# Creating DESDM DB tables

----

In [1]:
%matplotlib inline

In [2]:
import subprocess
import numpy as np
import fitsio
from matplotlib.mlab import rec2csv
import easyaccess as ea

## Create data for upload

In [3]:
size = (100)
x = np.random.normal(size=size)
y = np.random.randint(0,size,size)
z = y.astype(str)
data = np.rec.fromarrays([x,y,z],names=['xcol','ycol','zcol'])

In [4]:
fitstable = "mytable_fits" 
fitsfile = fitstable+'.fits' 
fitsio.write(fitsfile,data,clobber=True)

csvtable = "mytable_csv" 
csvfile = csvtable+'.csv' 
rec2csv(data,csvfile)

## Uploading with Default Typing

Upload the tables from the command line using ```easyaccess```. The datatype conversion is done by the ```easyaccess``` default mapping.

### Command-line upload

In [5]:
command = "easyaccess -q -lt %s"%fitsfile
print subprocess.check_output(command,shell=True)


 Table already exists. Table can be removed with:
  DROP TABLE MYTABLE_FITS;




In [6]:
command = "easyaccess -q -lt %s"%csvfile
print subprocess.check_output(command,shell=True)


 Table already exists. Table can be removed with:
  DROP TABLE MYTABLE_CSV;




Now check that the upload succeeded. Notice that the ```zcol``` in the CSV file has been uploaded as a ```NUMBER``` because the formatting was not preserved when writing to the CSV file. For the FITS file, the typing of this column was properly preserved as ```VARCHAR2```.

In [7]:
command = 'easyaccess -q -c "describe_table %s;"'%(csvtable)
print subprocess.check_output(command,shell=True)

command = 'easyaccess -q -c "select * from %s where rownum < 10;"'%(csvtable)
print subprocess.check_output(command,shell=True)


Description of MYTABLE_CSV commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL  BINARY_DOUBLE         8,,     None
2        YCOL         NUMBER       22,,0     None
3        ZCOL         NUMBER       22,,0     None




9 rows in 0.07 seconds

       XCOL  YCOL  ZCOL
1  1.700170    75    75
2 -1.166305    67    67
3  0.306881    12    12
4  1.808419    96    96
5 -0.564430    67    67
6  0.380664    76    76
7  0.401055     9     9
8 -0.318400    34    34
9 -0.755457    69    69




In [8]:
command = 'easyaccess -q -c "describe_table %s;"'%(fitstable)
print subprocess.check_output(command,shell=True)

command = 'easyaccess -q -c "select * from %s where rownum < 10;"'%(fitstable)
print subprocess.check_output(command,shell=True)


Description of MYTABLE_FITS commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL  BINARY_DOUBLE         8,,     None
2        YCOL         NUMBER        22,,     None
3        ZCOL       VARCHAR2        21,,     None




9 rows in 0.10 seconds

       XCOL  YCOL                   ZCOL
1  1.700170    75  75                   
2 -1.166305    67  67                   
3  0.306881    12  12                   
4  1.808419    96  96                   
5 -0.564430    67  67                   
6  0.380664    76  76                   
7  0.401055     9  9                    
8 -0.318400    34  34                   
9 -0.755457    69  69                   




### Python API upload

For more details on the ```easyaccess``` python API, see the notebook [here](https://github.com/mgckind/access_desdm/blob/master/easyaccess_api.ipynb). Below are the basics...

In [9]:
conn=ea.connect(section='dessci',quiet=True)

In [10]:
apitable = 'mytable_api'
conn.load_table(fitsfile,name=apitable)


 Table already exists. Table can be removed with:
  DROP TABLE MYTABLE_API;



In [11]:
conn.mytables()


List of my tables

        TABLE_NAME  SIZE_GBYTES
1     MYTABLE_FITS     0.000061
2  FGOTTENMETADATA     0.002930
3      MYTABLE_API     0.000061
4      MYTABLE_CSV     0.000061



Clean up a bit (the ```PURGE``` command is necessary to empty the "recycling bin").

In [17]:
for table in [csvtable,fitstable,apitable]:
    query = "DROP TABLE %s PURGE"%(table)
    print query
    conn.cursor().execute(query)

DROP TABLE mytable_csv PURGE
DROP TABLE mytable_fits PURGE
DROP TABLE mytable_api PURGE


DatabaseError: ORA-00942: table or view does not exist


## Upload with Custom Typing

The idea here is that we first create the table with a custom SQL command and then we upload the data to it.

In [13]:
sqltable = "mytable_sql"
sql = """
--- Custom Table Creation
CREATE TABLE %s (
xcol NUMBER(6,4) NOT NULL,
ycol BINARY_DOUBLE,
zcol VARCHAR2(5)
);
"""%sqltable

sqlfile = sqltable+".sql"
out = open(sqlfile,'w')
out.write(sql)
out.close()

In [18]:
# Create the table from the python API
query = conn.loadsql(sqlfile)

# This can also be done from the command line with:
# "easyaccess -q -loadsql %s"%sqlfile

In [19]:
conn.describe_table(sqltable)


Description of MYTABLE_SQL commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL         NUMBER      22,6,4     None
2        YCOL  BINARY_DOUBLE         8,,     None
3        ZCOL       VARCHAR2         5,,     None



Now we append the data file to the table that we just created.

In [20]:
conn.append_table(csvfile,sqltable)


 Inserted 100 rows and 3 columns into table MYTABLE_SQL in 0.73 seconds

 Table MYTABLE_SQL appended successfully.


In [22]:
conn.describe_table(sqltable)
conn.query_to_pandas("select * from %s where rownum < 10"%sqltable)


Description of MYTABLE_SQL commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL         NUMBER      22,6,4     None
2        YCOL  BINARY_DOUBLE         8,,     None
3        ZCOL       VARCHAR2         5,,     None



,XCOL,YCOL,ZCOL
0,-1.4702,65,65
1,-0.4718,98,98
2,-0.4847,16,16
3,-2.4167,24,24
4,-0.0351,70,70
5,0.6683,10,10
6,-1.4398,45,45
7,-0.4629,56,56
8,-1.9481,55,55
